## 44. サンプリング法

### <font color=blue>**1.** </font> A Tour of TensorFlow Probability

In [ ]:
## 出典： https://www.tensorflow.org/probability/examples/A_Tour_of_TensorFlow_Probability

## Copyright 2019 The TensorFlow Probability Authors.
## Licensed under the Apache License, Version 2.0 (the "License");

# In this Colab, we explore some of the fundamental features of TensorFlow Probability. 

In [ ]:
### Dependencies & Prerequisites

## Import

from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import tensorflow.compat.v2 as tf
import tensorflow_probability as tfp

In [ ]:
tf.enable_v2_behavior()

sns.reset_defaults()
sns.set_context(context='talk',font_scale=0.7)
plt.rcParams['image.cmap'] = 'viridis'

# %matplotlib inline

tfd = tfp.distributions
tfb = tfp.bijectors

In [ ]:
## Vectorization
# Vectorization makes things fast!
# It also means we think a lot about shapes
  
mats = tf.random.uniform(shape=[1000, 10, 10])
vecs = tf.random.uniform(shape=[1000, 10, 1])

def for_loop_solve():
  return np.array(
    [tf.linalg.solve(mats[i, ...], vecs[i, ...]) for i in range(1000)])

def vectorized_solve():
  return tf.linalg.solve(mats, vecs)

# Vectorization for the win!
%timeit for_loop_solve()
%timeit vectorized_solve()

In [ ]:
### Automatic Differentiation

a = tf.constant(np.pi)
b = tf.constant(np.e)

with tf.GradientTape() as tape:
  tape.watch([a, b])
  c = .5 * (a**2 + b**2)

grads = tape.gradient(c, [a, b])
print(grads[0])
print(grads[1])

In [ ]:
### A simple scalar-variate `Distribution`

## A standard normal

normal = tfd.Normal(loc=0., scale=1.)
print(normal)

In [ ]:
## Plot 1000 samples from a standard normal

samples = normal.sample(1000)
sns.distplot(samples)
plt.title("Samples from a standard Normal")
plt.show()

In [ ]:
## Compute the log_prob of a point in the event space of `normal`

normal.log_prob(0.)

In [ ]:
## Compute the log_prob of a few points

normal.log_prob([-1., 0., 1.])

In [ ]:
## A *batch* of scalar-variate `Distributions`
# Batches are like "vectorized" distributions: independent instances whose computations happen in parallel.

# Create a batch of 3 normals, and plot 1000 samples from each
normals = tfd.Normal([-2.5, 0., 2.5], 1.)  # The scale parameter broadacasts!
print("Batch shape:", normals.batch_shape)
print("Event shape:", normals.event_shape)

In [ ]:
# Samples' shapes go on the left!
samples = normals.sample(1000)
print("Shape of samples:", samples.shape)

In [ ]:
# Sample shapes can themselves be more complicated
print("Shape of samples:", normals.sample([10, 10, 10]).shape)

In [ ]:
# A batch of normals gives a batch of log_probs.
print(normals.log_prob([-2.5, 0., 2.5]))

In [ ]:
# The computation broadcasts, so a batch of normals applied to a scalar
# also gives a batch of log_probs.
print(normals.log_prob(0.))

In [ ]:
# Normal numpy-like broadcasting rules apply!
xs = np.linspace(-6, 6, 200)
try:
  normals.log_prob(xs)
except Exception as e:
  print("TFP error:", e.message)

In [ ]:
# That fails for the same reason this does:
try:
  np.zeros(200) + np.zeros(3)
except Exception as e:
  print("Numpy error:", e)

In [ ]:
# But this would work:
a = np.zeros([200, 1]) + np.zeros(3)
print("Broadcast shape:", a.shape)

In [ ]:
# And so will this!
xs = np.linspace(-6, 6, 200)[..., np.newaxis]
# => shape = [200, 1]

lps = normals.log_prob(xs)
print("Broadcast log_prob shape:", lps.shape)

In [ ]:
# Summarizing visually
for i in range(3):
  sns.distplot(samples[:, i], kde=False, norm_hist=True)
  
plt.plot(np.tile(xs, 3), normals.prob(xs), c='k', alpha=.5)
plt.title("Samples from 3 Normals, and their PDF's")
plt.show()

In [ ]:
## A vector-variate `Distribution`

mvn = tfd.MultivariateNormalDiag(loc=[0., 0.], scale_diag = [1., 1.])
print("Batch shape:", mvn.batch_shape)
print("Event shape:", mvn.event_shape)

In [ ]:
samples = mvn.sample(1000)
print("Samples shape:", samples.shape)

In [ ]:
g = sns.jointplot(samples[:, 0], samples[:, 1], kind='scatter')
plt.show()

In [ ]:
## A matrix-variate `Distribution`

lkj = tfd.LKJ(dimension=10, concentration=[1.5, 3.0])
print("Batch shape: ", lkj.batch_shape)
print("Event shape: ", lkj.event_shape)

In [ ]:
samples = lkj.sample()
print("Samples shape: ", samples.shape)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 4))
sns.heatmap(samples[0, ...], ax=axes[0], cbar=False)
sns.heatmap(samples[1, ...], ax=axes[1], cbar=False)
fig.tight_layout()
plt.show()

In [ ]:
## Gaussian Processes

kernel = tfp.math.psd_kernels.ExponentiatedQuadratic()
xs = np.linspace(-5., 5., 200).reshape([-1, 1])
gp = tfd.GaussianProcess(kernel, index_points=xs)
print("Batch shape:", gp.batch_shape)
print("Event shape:", gp.event_shape)

In [ ]:
upper, lower = gp.mean() + [2 * gp.stddev(), -2 * gp.stddev()]
plt.figure(figsize=(8,6))
plt.plot(xs, gp.mean())
plt.fill_between(xs[..., 0], upper, lower, color='k', alpha=.1)
for _ in range(5):
  plt.plot(xs, gp.sample(), c='r', alpha=.3)
plt.title(r"GP prior mean, $2\sigma$ intervals, and samples")
plt.show()

#    *** Bonus question ***
# Why do so many of these functions lie outside the 95% intervals?

In [ ]:
## GP Regression

# Suppose we have some observed data
obs_x = [[-3.], [0.], [2.]]  # Shape 3x1 (3 1-D vectors)
obs_y = [3., -2., 2.]        # Shape 3   (3 scalars)

gprm = tfd.GaussianProcessRegressionModel(kernel, xs, obs_x, obs_y)

In [ ]:
upper, lower = gprm.mean() + [2 * gprm.stddev(), -2 * gprm.stddev()]
plt.figure(figsize=(8,6))
plt.plot(xs, gprm.mean())
plt.fill_between(xs[..., 0], upper, lower, color='k', alpha=.1)
for _ in range(5):
  plt.plot(xs, gprm.sample(), c='r', alpha=.3)
plt.scatter(obs_x, obs_y, c='k', zorder=3)
plt.title(r"GP posterior mean, $2\sigma$ intervals, and samples")
plt.show()

<font size=5>Bijectors</font>

Bijectors represent (mostly) invertible, smooth functions. They can be used to transform distributions, preserving the ability to take samples and compute log_probs. They can be in the `tfp.bijectors` module.

Each bijector implements at least 3 methods: 
  * `forward`,
  * `inverse`, and
  * (at least) one of `forward_log_det_jacobian` and `inverse_log_det_jacobian`.

With these ingredients, we can transform a distribution and still get samples and log probs from the result!

In Math, somewhat sloppily

* $X$ is a random variable with pdf $p(x)$
* $g$ is a smooth, invertible function on the space of $X$'s
* $Y = g(X)$ is a new, transformed random variable
* $p(Y=y) = p(X=g^{-1}(y)) \cdot |\nabla g^{-1}(y)|$

Caching

Bijectors also cache the forward and inverse computations, and log-det-Jacobians, which allows us to save
repeating potentially very expensive operations!

In [ ]:
## A Simple `Bijector`

normal_cdf = tfp.bijectors.NormalCDF()
xs = np.linspace(-4., 4., 200)
plt.plot(xs, normal_cdf.forward(xs))
plt.show()

In [ ]:
plt.plot(xs, normal_cdf.forward_log_det_jacobian(xs, event_ndims=0))
plt.show()

In [ ]:
## A `Bijector` transforming a `Distribution`

exp_bijector = tfp.bijectors.Exp()
log_normal = exp_bijector(tfd.Normal(0., .5))

samples = log_normal.sample(1000)
xs = np.linspace(1e-10, np.max(samples), 200)
sns.distplot(samples, norm_hist=True, kde=False)
plt.plot(xs, log_normal.prob(xs), c='k', alpha=.75)
plt.show()

In [ ]:
## Batching `Bijectors`

# Create a batch of bijectors of shape [3,]
softplus = tfp.bijectors.Softplus(
  hinge_softness=[1., .5, .1])
print("Hinge softness shape:", softplus.hinge_softness.shape)

In [ ]:
# For broadcasting, we want this to be shape [200, 1]
xs = np.linspace(-4., 4., 200)[..., np.newaxis]
ys = softplus.forward(xs)
print("Forward shape:", ys.shape)

In [ ]:
# Visualization
lines = plt.plot(np.tile(xs, 3), ys)
for line, hs in zip(lines, softplus.hinge_softness):
  line.set_label("Softness: %1.1f" % hs)
plt.legend()
plt.show()

In [ ]:
## Caching

# This bijector represents a matrix outer product on the forward pass,
# and a cholesky decomposition on the inverse pass. The latter costs O(N^3)!
bij = tfb.CholeskyOuterProduct()

size = 2500
# Make a big, lower-triangular matrix
big_lower_triangular = tf.eye(size)
# Squaring it gives us a positive-definite matrix
big_positive_definite = bij.forward(big_lower_triangular)

# Caching for the win!
%timeit bij.inverse(big_positive_definite)
%timeit tf.linalg.cholesky(big_positive_definite)

### <font color=blue>**2.** </font> ライブラリの使用例

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm

import warnings
warnings.simplefilter('ignore', FutureWarning)

In [ ]:
## Metropolis Hastings

tfd = tfp.distributions

dtype = np.float32

target = tfd.Normal(loc=dtype(0), scale=dtype(1))


rwm_kernel = tfp.mcmc.RandomWalkMetropolis(target.log_prob)

samples = tfp.mcmc.sample_chain(
  num_results=1000,
  current_state=dtype(1),
  kernel=rwm_kernel,
  num_burnin_steps=500,
  trace_fn=None)

sample_mean = tf.math.reduce_mean(samples, axis=0)
sample_std = tf.sqrt(
    tf.math.reduce_mean(
        tf.math.squared_difference(samples, sample_mean),
        axis=0))

print('予測平均: {}'.format(sample_mean))
print('予測標準偏差: {}'.format(sample_std))

In [ ]:
plt.figure(figsize=(10,8))
x = np.linspace(-5, 5, 1000)
plt.plot(x, norm.pdf(x))
sns.distplot(samples)
plt.show()

In [ ]:
## Hamiltonian Monte Carlo

tfd = tfp.distributions

dtype = np.float32

target = tfd.Normal(loc=dtype(0), scale=dtype(1))


hmc_kernel = tfp.mcmc.HamiltonianMonteCarlo(
  target_log_prob_fn=target.log_prob,
  step_size=np.float64(.5),
  num_leapfrog_steps=np.float64(2))

samples = tfp.mcmc.sample_chain(
  num_results=1000,
  current_state=dtype(1),
  kernel=hmc_kernel,
  num_burnin_steps=500,
  trace_fn=None)

sample_mean = tf.math.reduce_mean(samples, axis=0)
sample_std = tf.sqrt(
    tf.math.reduce_mean(
        tf.math.squared_difference(samples, sample_mean),
        axis=0))

print('予測平均: {}'.format(sample_mean))
print('予測標準偏差: {}'.format(sample_std))

In [ ]:
plt.figure(figsize=(10,8))
x = np.linspace(-5, 5, 1000)
plt.plot(x, norm.pdf(x))
sns.distplot(samples)
plt.show()

In [ ]:
## Simple Step Size Adaptation

tfd = tfp.distributions

dtype = np.float32

target = tfd.Normal(loc=dtype(0), scale=dtype(1))


hmc_kernel = tfp.mcmc.HamiltonianMonteCarlo(
  target_log_prob_fn=target.log_prob,
  step_size=np.float64(.5),
  num_leapfrog_steps=np.float64(2))

sssa_kernel = tfp.mcmc.SimpleStepSizeAdaptation(
    inner_kernel=hmc_kernel, num_adaptation_steps=int(1000 * 0.8),
    target_accept_prob=0.66)

samples = tfp.mcmc.sample_chain(
  num_results=1000,
  current_state=dtype(1),
  kernel=sssa_kernel,
  num_burnin_steps=500,
  trace_fn=None)

sample_mean = tf.math.reduce_mean(samples, axis=0)
sample_std = tf.sqrt(
    tf.math.reduce_mean(
        tf.math.squared_difference(samples, sample_mean),
        axis=0))

print('予測平均: {}'.format(sample_mean))
print('予測標準偏差: {}'.format(sample_std))

In [ ]:
plt.figure(figsize=(10,8))
x = np.linspace(-5, 5, 1000)
plt.plot(x, norm.pdf(x))
sns.distplot(samples)
plt.show()

In [ ]:
## Dual Averaging Step Size Adaptation

tfd = tfp.distributions

dtype = np.float32

target = tfd.Normal(loc=dtype(0), scale=dtype(1))


hmc_kernel = tfp.mcmc.HamiltonianMonteCarlo(
  target_log_prob_fn=target.log_prob,
  step_size=np.float64(.5),
  num_leapfrog_steps=np.float64(2))

dassa_kernel = tfp.mcmc.DualAveragingStepSizeAdaptation(
    inner_kernel=hmc_kernel, num_adaptation_steps=int(500),
    target_accept_prob=0.66)

samples = tfp.mcmc.sample_chain(
  num_results=1000,
  current_state=dtype(1),
  kernel=dassa_kernel,
  num_burnin_steps=500,
  trace_fn=None)

sample_mean = tf.math.reduce_mean(samples, axis=0)
sample_std = tf.sqrt(
    tf.math.reduce_mean(
        tf.math.squared_difference(samples, sample_mean),
        axis=0))

print('予測平均: {}'.format(sample_mean))
print('予測標準偏差: {}'.format(sample_std))

In [ ]:
plt.figure(figsize=(10,8))
x = np.linspace(-5, 5, 1000)
plt.plot(x, norm.pdf(x))
sns.distplot(samples)
plt.show()

In [ ]:
## Metropolis Adjusted Langevin Algorithm

tfd = tfp.distributions

dtype = np.float32

target = tfd.Normal(loc=dtype(0), scale=dtype(1))


mala_kernel = tfp.mcmc.MetropolisAdjustedLangevinAlgorithm(
        target_log_prob_fn=target.log_prob,
        step_size=0.75)
samples = tfp.mcmc.sample_chain(
  num_results=1000,
  current_state=dtype(1),
  kernel=mala_kernel,
  num_burnin_steps=500,
  trace_fn=None)

sample_mean = tf.math.reduce_mean(samples, axis=0)
sample_std = tf.sqrt(
    tf.math.reduce_mean(
        tf.math.squared_difference(samples, sample_mean),
        axis=0))

print('予測平均: {}'.format(sample_mean))
print('予測標準偏差: {}'.format(sample_std))

In [ ]:
plt.figure(figsize=(10,8))
x = np.linspace(-5, 5, 1000)
plt.plot(x, norm.pdf(x))
sns.distplot(samples)
plt.show()

#### <font color=red>task : </font> ガンマ分布の場合について、同様に上記5つの各メソッドによりサンプリングし図示してみる

### <font color=blue>**3.** </font> ギブスサンプリング（MCMC : Markov chain Monte Carlo methods）による画像のノイズ除去

In [ ]:
## 出典: https://ichi.pro/gibusu-sanpuringu-mcmc-niyoru-gazo-no-noizu-jokyo-19602944876883

In [ ]:
import matplotlib.pyplot as plt
import cv2

In [ ]:
img = cv2.imread("/content/img.png", 0)
img_noisy = cv2.imread("/content/img_noisy.png", 0)

plt.figure(figsize=(16,7))
plt.subplot(1,2,1)
plt.imshow(img, cmap = 'gray')
plt.subplot(1,2,2)
plt.imshow(img_noisy, cmap = 'gray')
plt.show()

In [ ]:
import math
import numpy as np
import time

In [ ]:
def load_image(filename):
  ## PNG画像をnumpy配列に読み取り
  my_img = plt.imread(filename)
  
  ## グレースケールに変換
  img_gray = np.dot(my_img[..., :3], [0.2989, 0.5870, 0.1140])
  
  ## ピクセルを{-1、1}に再スケーリング
  img_gray = np.where(img_gray > 0.5, 1, -1)
  
  ## 各ピクセルの隣接ピクセルを検索するときにコーナーケースを処理できるように、エッジに0個のパディングを追加
  img_padded = np.zeros([img_gray.shape[0] + 2, img_gray.shape[1] + 2])
  img_padded[1:-1, 1:-1] = img_gray
  return img_padded

$\log{P(Y|X)} = \log{P(X|Y)} + \log{P(Y)} - \log{P(X)}$

$\displaystyle P(Y, X) = \dfrac{1}{Z} \exp \left( \eta \sum_{i=1}^{N} \sum_{j=1}^{M}{x_{ij}y_{ij}} + \beta \sum_{i'j' \in N(ij)}^{} {y_{ij}y_{i'j'}} \right)$

In [ ]:
def sample_y(i, j, Y, X):
  ## 行と列のインデックス i と j
  ## 復元された画像配列Y
  ## ノイズの多い画像配列X

  ## yij の近傍 yij_neighbors を検索し、条件付き確率P（yij = 1 | yij_neighbors）を計算
  markov_blanket = [Y[i - 1, j], Y[i, j - 1], Y[i, j + 1], Y[i + 1, j], X[i, j]]
  w = ETA * markov_blanket[-1] + BETA * sum(markov_blanket[:4])

  ## 条件付き確率でサンプリングされた yij の値（1または-1）を返す
  prob = 1 / (1 + math.exp(-2*w))
  return (np.random.rand() < prob) * 2 - 1

$\displaystyle P(y_{ij} = 1 | y_{N(ij)}, x_{i, j}) = \cdots = \dfrac{1}{1 + \exp (-2w_{ij})}$

$\displaystyle w_{ij} = \eta x_{ij} + \beta \sum_{N(ij)} y_{N(ij)}$

In [ ]:
def get_posterior(filename, burn_in_steps, total_samples, logfile):
  ## ノイズの多い画像Xをロード
  X = load_image(filename)
  
  posterior = np.zeros(X.shape)
  print("img shape: {}".format(X.shape))
  
  ## 復元された画像Yをランダムに初期化
  Y = np.random.choice([1, -1], size=X.shape)
  energy_list = list()
  
  ## Yをサンプリングし、事後確率P（Y | Y_neighbor）を計算
  for step in range(burn_in_steps + total_samples):
    if step % 10 == 0:
      print("{}th step start".format(step+1))
    for i in range(1, Y.shape[0]-1):
      for j in range(1, Y.shape[1]-1):
        ## Yの各ピクセルをサンプリング
        y = sample_y(i, j, Y, X)

        ## サンプリングされた値でYを更新
        Y[i, j] = y

        ## バーンイン期間が終了すると、Yのyijについて、yij = 1というイベントの発生総数を合計
        if y == 1 and step >= burn_in_steps:
          posterior[i, j] += 1
    ## 収束を視覚化できるように、エネルギーを追跡
    energy = -np.sum(np.multiply(Y, X))*ETA-(np.sum(np.multiply(Y[:-1], Y[1:]))+np.sum(np.multiply(Y[:, :-1], Y[:, 1:])))*BETA
    if step < burn_in_steps:
      energy_list.append(str(step) + "\t" + str(energy) + "\tB")
    else:
      energy_list.append(str(step) + "\t" + str(energy) + "\tS")
  ## サンプリングが完了したら、モンテカルロ法を使用して事後確率を取得
  ## 事後確率は、基本的にYの集計値を合計サンプル数で除算
  posterior = posterior / total_samples

  file = open(logfile, 'w')
  for element in energy_list:
    file.writelines(element)
    file.write('\n')
  file.close()
  return posterior

In [ ]:
## 入力関数

def denoise_image(filename, burn_in_steps, total_samples, logfile):
  ## 推定事後確率p（Y = 1 | Y_neighbor）を取得
  posterior = get_posterior(filename, burn_in_steps, total_samples, logfile=logfile)
  
  denoised = np.zeros(posterior.shape, dtype=np.float64)
  
  ## しきい値を0.5に設定すると、復元された画像配列Yを後方から取得
  denoised[posterior > 0.5] = 1
  
  ## 画像配列のエッジを取り除いて返す
  return denoised[1:-1, 1:-1]

In [ ]:
def plot_energy(filename):
  x = np.genfromtxt(filename, dtype=None, encoding='utf8')
  its, energies, phases = zip(*x)
  its = np.asarray(its)
  energies = np.asarray(energies)
  phases = np.asarray(phases)
  burn_mask = (phases == 'B')
  samp_mask = (phases == 'S')
  assert np.sum(burn_mask) + np.sum(samp_mask) == len(x), 'Found bad phase'
  its_burn, energies_burn = its[burn_mask], energies[burn_mask]
  its_samp, energies_samp = its[samp_mask], energies[samp_mask]
  p1, = plt.plot(its_burn, energies_burn, 'r')
  p2, = plt.plot(its_samp, energies_samp, 'b')
  plt.title("energy")
  plt.xlabel('iteration number')
  plt.ylabel('energy')
  plt.legend([p1, p2], ['burn in', 'sampling'])
  plt.show()  ###

  plt.savefig('%s.png' % filename[:-4])
  plt.close()

In [ ]:
def save_image(denoised_image):
  plt.figure(figsize=(8,7))  ###
  plt.imshow(denoised_image, cmap='gray')
  plt.title("denoised image")
  plt.show()  ###

  plt.savefig('/content/denoise_image.png') ###
  plt.close()

In [ ]:
## ハイパーパラメータ η と β
ETA = 1
BETA = 1

## サンプリングステップ
total_samples = 180  ###

## 書き込みステップ
burn_in_steps = 20 ###

logfile = "/content/log_energy.txt" ###

In [ ]:
time1 = time.time()
denoised_img = denoise_image("/content/img_noisy.png",  ###
                             burn_in_steps = burn_in_steps,
                             total_samples = total_samples, 
                             logfile = logfile
                             )
print("total time: {}".format(time.time() - time1))
save_image(denoised_img)

In [ ]:
# log = open("/content/log_energy.txt")
plot_energy(logfile)

### <font color=blue>**4.** </font> Eight schools

The eight schools problem ([Rubin 1981](https://www.jstor.org/stable/1164617)) considers the effectiveness of SAT coaching programs conducted in parallel at eight schools. It has become a classic problem ([Bayesian Data Analysis](http://www.stat.columbia.edu/~gelman/book/), [Stan](https://github.com/stan-dev/rstan/wiki/RStan-Getting-Started)) that illustrates the usefulness of hierarchical modeling for sharing information between exchangeable groups.

The implemention below is an adaptation of an Edward 1.0 [tutorial](https://github.com/blei-lab/edward/blob/master/notebooks/eight_schools.ipynb).

In [ ]:
## Copyright 2018 The TensorFlow Probability Authors.
## Licensed under the Apache License, Version 2.0 (the "License");

# 日本人による解説記事：https://qiita.com/aoki-h/items/b8281823146b0e6c3ac2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import tensorflow.compat.v2 as tf
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd
import warnings

tf.enable_v2_behavior()

plt.style.use("ggplot")
warnings.filterwarnings('ignore')

In [ ]:
'''The Data
From Bayesian Data Analysis, section 5.5 (Gelman et al. 2013):
 A study was performed for the Educational Testing Service to analyze the effects of special
 coaching programs for SAT-V (Scholastic Aptitude Test-Verbal) in each of eight high schools.
 The outcome variable in each study was the score on a special administration of the SAT-V, 
 a standardized multiple choice test administered by the Educational Testing Service and used to 
 help colleges make admissions decisions; the scores can vary between 200 and 800, 
 with mean about 500 and standard deviation about 100. 
 The SAT examinations are designed to be resistant to short-term efforts directed specifically toward 
 improving performance on the test; instead they are designed to reflect knowledge acquired and abilities 
 developed over many years of education. Nevertheless, each of the eight schools in this study considered 
 its short-term coaching program to be very successful at increasing SAT scores. 
 Also, there was no prior reason to believe that any of the eight programs was more effective than any other 
 or that some were more similar in effect to each other than to any other.

For each of the eight schools ($J = 8$), we have an estimated treatment effect $y_j$ and a standard error of the effect estimate $\sigma_j$. 
The treatment effects in the study were obtained by a linear regression on the treatment group using PSAT-M and PSAT-V scores as control variables. 
As there was no prior belief that any of the schools were more or less similar or that any of the coaching programs would be more effective, 
we can consider the treatment effects as [exchangeable](https://en.wikipedia.org/wiki/Exchangeable_random_variables).
'''

In [ ]:
num_schools = 8  # number of schools
treatment_effects = np.array(
    [28, 8, -3, 7, -1, 1, 18, 12], dtype=np.float32)  # treatment effects
treatment_stddevs = np.array(
    [15, 10, 16, 11, 9, 11, 10, 18], dtype=np.float32)  # treatment SE

fig, ax = plt.subplots()
plt.bar(range(num_schools), treatment_effects, yerr=treatment_stddevs)
plt.title("8 Schools treatment effects")
plt.xlabel("School")
plt.ylabel("Treatment effect")
fig.set_size_inches(10, 8)
plt.show()

In [ ]:
## Model
# To capture the data, we use a hierarchical normal model. It follows the generative process,


$$
\begin{align*}
\mu &\sim \text{Normal}(\text{loc}{=}0,\ \text{scale}{=}10) \\
\log\tau &\sim \text{Normal}(\text{loc}{=}5,\ \text{scale}{=}1) \\
\text{for } & i=1\ldots 8:\\
& \theta_i \sim \text{Normal}\left(\text{loc}{=}\mu,\ \text{scale}{=}\tau \right) \\
& y_i \sim \text{Normal}\left(\text{loc}{=}\theta_i,\ \text{scale}{=}\sigma_i \right)
\end{align*}
$$

where $\mu$ represents the prior average treatment effect and $\tau$ controls how much variance there is between schools. The $y_i$ and $\sigma_i$ are observed. As $\tau \rightarrow \infty$, the model approaches the no-pooling model, i.e., each of the school treatment effect estimates are allowed to be more independent. As $\tau \rightarrow 0$, the model approaches the complete-pooling model, i.e., all of the school treatment effects are closer to the group average $\mu$. To restrict the standard deviation to be positive, we draw $\tau$ from a lognormal distribution (which is equivalent to drawing $log(\tau)$ from a normal distribution).


Following [Diagnosing Biased Inference with Divergences](http://mc-stan.org/users/documentation/case-studies/divergences_and_bias.html), we transform the model above into an equivalent non-centered model:

$$
\begin{align*}
\mu &\sim \text{Normal}(\text{loc}{=}0,\ \text{scale}{=}10) \\
\log\tau &\sim \text{Normal}(\text{loc}{=}5,\ \text{scale}{=}1) \\
\text{for } & i=1\ldots 8:\\
& \theta_i' \sim \text{Normal}\left(\text{loc}{=}0,\ \text{scale}{=}1 \right) \\
& \theta_i = \mu + \tau \theta_i' \\
& y_i \sim \text{Normal}\left(\text{loc}{=}\theta_i,\ \text{scale}{=}\sigma_i \right) 
\end{align*}
$$

We reify this model as a [JointDistributionSequential](https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/JointDistributionSequential) instance:

In [ ]:
model = tfd.JointDistributionSequential([
  tfd.Normal(loc=0., scale=10., name="avg_effect"),  # `mu` above
  tfd.Normal(loc=5., scale=1., name="avg_stddev"),  # `log(tau)` above
  tfd.Independent(tfd.Normal(loc=tf.zeros(num_schools),
                             scale=tf.ones(num_schools),
                             name="school_effects_standard"),  # `theta_prime` 
                  reinterpreted_batch_ndims=1),
  lambda school_effects_standard, avg_stddev, avg_effect: (
      tfd.Independent(tfd.Normal(loc=(avg_effect[..., tf.newaxis] +
                                      tf.exp(avg_stddev[..., tf.newaxis]) *
                                      school_effects_standard),  # `theta` above
                                 scale=treatment_stddevs),
                      name="treatment_effects",  # `y` above
                      reinterpreted_batch_ndims=1))
])

In [ ]:
def target_log_prob_fn(avg_effect, avg_stddev, school_effects_standard):
  """Unnormalized target density as a function of states."""
  return model.log_prob((
      avg_effect, avg_stddev, school_effects_standard, treatment_effects))

In [ ]:
## Bayesian Inference
# Given data, we perform Hamiltonian Monte Carlo (HMC) to calculate the posterior distribution over the model's parameters.

In [ ]:
num_results = 5000
num_burnin_steps = 3000

In [ ]:
# Improve performance by tracing the sampler using `tf.function`
# and compiling it using XLA.
@tf.function(autograph=False, experimental_compile=True)
def do_sampling():
  return tfp.mcmc.sample_chain(
      num_results=num_results,
      num_burnin_steps=num_burnin_steps,
      current_state=[
          tf.zeros([], name='init_avg_effect'),
          tf.zeros([], name='init_avg_stddev'),
          tf.ones([num_schools], name='init_school_effects_standard'),
      ],
      kernel=tfp.mcmc.HamiltonianMonteCarlo(
          target_log_prob_fn=target_log_prob_fn,
          step_size=0.4,
          num_leapfrog_steps=3))

In [ ]:
states, kernel_results = do_sampling()

In [ ]:
avg_effect, avg_stddev, school_effects_standard = states

In [ ]:
school_effects_samples = (
    avg_effect[:, np.newaxis] +
    np.exp(avg_stddev)[:, np.newaxis] * school_effects_standard)

num_accepted = np.sum(kernel_results.is_accepted)
print('Acceptance rate: {}'.format(num_accepted / num_results))

In [ ]:
fig, axes = plt.subplots(8, 2, sharex='col', sharey='col')
fig.set_size_inches(12, 10)
for i in range(num_schools):
  axes[i][0].plot(school_effects_samples[:,i].numpy())
  axes[i][0].title.set_text("School {} treatment effect chain".format(i))
  sns.kdeplot(school_effects_samples[:,i].numpy(), ax=axes[i][1], shade=True)
  axes[i][1].title.set_text("School {} treatment effect distribution".format(i))
axes[num_schools - 1][0].set_xlabel("Iteration")
axes[num_schools - 1][1].set_xlabel("School effect")
fig.tight_layout()
plt.show()

In [ ]:
print("E[avg_effect] = {}".format(np.mean(avg_effect)))
print("E[avg_stddev] = {}".format(np.mean(avg_stddev)))
print("\nE[school_effects_standard] =")
print(np.mean(school_effects_standard[:, ]))
print("\nE[school_effects] =")
print(np.mean(school_effects_samples[:, ], axis=0))

In [ ]:
# Compute the 95% interval for school_effects
school_effects_low = np.array([
    np.percentile(school_effects_samples[:, i], 2.5) for i in range(num_schools)
])

school_effects_med = np.array([
    np.percentile(school_effects_samples[:, i], 50) for i in range(num_schools)
])

school_effects_hi = np.array([
    np.percentile(school_effects_samples[:, i], 97.5)
    for i in range(num_schools)
])

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, sharex=True)
ax.scatter(np.array(range(num_schools)), school_effects_med, color='red', s=60)
ax.scatter(
    np.array(range(num_schools)) + 0.1, treatment_effects, color='blue', s=60)

plt.plot([-0.2, 7.4], [np.mean(avg_effect),
                       np.mean(avg_effect)], 'k', linestyle='--')

ax.errorbar(
    np.array(range(8)),
    school_effects_med,
    yerr=[
        school_effects_med - school_effects_low,
        school_effects_hi - school_effects_med
    ],
    fmt='none')

ax.legend(('avg_effect', 'HMC', 'Observed effect'), fontsize=14)

plt.xlabel('School')
plt.ylabel('Treatment effect')
plt.title('HMC estimated school treatment effects vs. observed data')
fig.set_size_inches(10, 8)
plt.show()

In [ ]:
## We can observe the shrinkage toward the group `avg_effect` above.

In [ ]:
print("Inferred posterior mean: {0:.2f}".format(
    np.mean(school_effects_samples[:,])))
print("Inferred posterior mean se: {0:.2f}".format(
    np.std(school_effects_samples[:,])))

<font size=5>Criticism</font>

To get the posterior predictive distribution, i.e., a model of new data $y^*$ given the observed data $y$:

$$ p(y^*|y) \propto \int_\theta p(y^* | \theta)p(\theta |y)d\theta$$

we override the values of the random variables in the model to set them to the mean of the posterior distribution, and sample from that model to generate new data $y^*$.

In [ ]:
sample_shape = [5000]

_, _, _, predictive_treatment_effects = model.sample(
    value=(tf.broadcast_to(np.mean(avg_effect, 0), sample_shape),
           tf.broadcast_to(np.mean(avg_stddev, 0), sample_shape),
           tf.broadcast_to(np.mean(school_effects_standard, 0),
                           sample_shape + [num_schools]),
           None))

In [ ]:
fig, axes = plt.subplots(4, 2, sharex=True, sharey=True)
fig.set_size_inches(12, 10)
fig.tight_layout()
for i, ax in enumerate(axes):
  sns.kdeplot(predictive_treatment_effects[:, 2*i].numpy(),
              ax=ax[0], shade=True)
  ax[0].title.set_text(
      "School {} treatment effect posterior predictive".format(2*i))
  sns.kdeplot(predictive_treatment_effects[:, 2*i + 1].numpy(),
              ax=ax[1], shade=True)
  ax[1].title.set_text(
      "School {} treatment effect posterior predictive".format(2*i + 1))
plt.show()

In [ ]:
# The mean predicted treatment effects for each of the eight schools.
prediction = np.mean(predictive_treatment_effects, axis=0)

We can look at the residuals between the treatment effects data and the predictions of the model posterior. These correspond with the plot above which shows the shrinkage of the estimated effects toward the population average.

In [ ]:
treatment_effects - prediction

Because we have a distribution of predictions for each school, we can consider the distribution of residuals as well.

In [ ]:
residuals = treatment_effects - predictive_treatment_effects

In [ ]:
fig, axes = plt.subplots(4, 2, sharex=True, sharey=True)
fig.set_size_inches(12, 10)
fig.tight_layout()
for i, ax in enumerate(axes):
  sns.kdeplot(residuals[:, 2*i].numpy(), ax=ax[0], shade=True)
  ax[0].title.set_text(
      "School {} treatment effect residuals".format(2*i))
  sns.kdeplot(residuals[:, 2*i + 1].numpy(), ax=ax[1], shade=True)
  ax[1].title.set_text(
      "School {} treatment effect residuals".format(2*i + 1))
plt.show()

Acknowledgements

This tutorial was originally written in Edward 1.0 ([source](https://github.com/blei-lab/edward/blob/master/notebooks/eight_schools.ipynb)). We thank all contributors to writing and revising that version.

### <font color=blue>**5.** </font> A Tour of TensorFlow Probability　　続き

In [ ]:
### Dependencies & Prerequisites

## Import

from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import tensorflow.compat.v2 as tf
import tensorflow_probability as tfp

In [ ]:
tf.enable_v2_behavior()

sns.reset_defaults()
sns.set_context(context='talk',font_scale=0.7)
plt.rcParams['image.cmap'] = 'viridis'

# %matplotlib inline

tfd = tfp.distributions
tfb = tfp.bijectors

In [ ]:
## MCMC

# TFP has built in support for some standard Markov chain Monte Carlo algorithms, including Hamiltonian Monte Carlo.

In [ ]:
## Generate a data set

# Generate some data
def f(x, w):
  # Pad x with 1's so we can add bias via matmul
  x = tf.pad(x, [[1, 0], [0, 0]], constant_values=1)
  linop = tf.linalg.LinearOperatorFullMatrix(w[..., np.newaxis])
  result = linop.matmul(x, adjoint=True)
  return result[..., 0, :]

In [ ]:
num_features = 2
num_examples = 50
noise_scale = .5
true_w = np.array([-1., 2., 3.])

xs = np.random.uniform(-1., 1., [num_features, num_examples])
ys = f(xs, true_w) + np.random.normal(0., noise_scale, size=num_examples)

In [ ]:
# Visualize the data set
plt.scatter(*xs, c=ys, s=100, linewidths=0)

grid = np.meshgrid(*([np.linspace(-1, 1, 100)] * 2))
xs_grid = np.stack(grid, axis=0)
fs_grid = f(xs_grid.reshape([num_features, -1]), true_w)
fs_grid = np.reshape(fs_grid, [100, 100])
plt.colorbar()
plt.contour(xs_grid[0, ...], xs_grid[1, ...], fs_grid, 20, linewidths=1)
plt.show()

In [ ]:
## Define our joint log-prob function
# The unnormalized posterior is the result of closing over the data to form a
# partial application of the joint log prob.

# Define the joint_log_prob function, and our unnormalized posterior.
def joint_log_prob(w, x, y):
  # Our model in maths is
  #   w ~ MVN([0, 0, 0], diag([1, 1, 1]))
  #   y_i ~ Normal(w @ x_i, noise_scale),  i=1..N

  rv_w = tfd.MultivariateNormalDiag(
    loc=np.zeros(num_features + 1),
    scale_diag=np.ones(num_features + 1))

  rv_y = tfd.Normal(f(x, w), noise_scale)
  return (rv_w.log_prob(w) +
          tf.reduce_sum(rv_y.log_prob(y), axis=-1))

In [ ]:
# Create our unnormalized target density by currying x and y from the joint.
def unnormalized_posterior(w):
  return joint_log_prob(w, xs, ys)

In [ ]:
## Build HMC TransitionKernel and call sample_chain

# Create an HMC TransitionKernel
hmc_kernel = tfp.mcmc.HamiltonianMonteCarlo(
  target_log_prob_fn=unnormalized_posterior,
  step_size=np.float64(.1),
  num_leapfrog_steps=2)

In [ ]:
# We wrap sample_chain in tf.function, telling TF to precompile a reusable
# computation graph, which will dramatically improve performance.
@tf.function
def run_chain(initial_state, num_results=1000, num_burnin_steps=500):
  return tfp.mcmc.sample_chain(
    num_results=num_results,
    num_burnin_steps=num_burnin_steps,
    current_state=initial_state,
    kernel=hmc_kernel,
    trace_fn=lambda current_state, kernel_results: kernel_results)

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

In [ ]:
initial_state = np.zeros(num_features + 1)
samples, kernel_results = run_chain(initial_state)
print("Acceptance rate:", kernel_results.is_accepted.numpy().mean())

*That's not great! We'd like an acceptance rate closer to .65.*

(see ["Optimal Scaling for Various Metropolis-Hastings Algorithms"](https://projecteuclid.org/euclid.ss/1015346320), Roberts & Rosenthal, 2001)


In [ ]:
## Adaptive step sizes

# We can wrap our HMC TransitionKernel in a `SimpleStepSizeAdaptation` "meta-kernel", 
# which will apply some (rather simple heuristic) logic to adapt the HMC step size during burnin.
# We allot 80% of burnin for adapting step size, and then let the remaining 20% go just to mixing.

In [ ]:
# Apply a simple step size adaptation during burnin
@tf.function
def run_chain(initial_state, num_results=1000, num_burnin_steps=500):
  adaptive_kernel = tfp.mcmc.SimpleStepSizeAdaptation(
      hmc_kernel,
      num_adaptation_steps=int(.8 * num_burnin_steps),
      target_accept_prob=np.float64(.65))

  return tfp.mcmc.sample_chain(
    num_results=num_results,
    num_burnin_steps=num_burnin_steps,
    current_state=initial_state,
    kernel=adaptive_kernel,
    trace_fn=lambda cs, kr: kr)

In [ ]:
samples, kernel_results = run_chain(
  initial_state=np.zeros(num_features+1))
print("Acceptance rate:", kernel_results.inner_results.is_accepted.numpy().mean())

In [ ]:
# Trace plots
colors = ['b', 'g', 'r']
for i in range(3):
  plt.plot(samples[:, i], c=colors[i], alpha=.3)
  plt.hlines(true_w[i], 0, 1000, zorder=4, color=colors[i], label="$w_{}$".format(i))
plt.legend(loc='upper right')
plt.show()

In [ ]:
import warnings
warnings.simplefilter('ignore', FutureWarning)

In [ ]:
# Histogram of samples
for i in range(3):
  sns.distplot(samples[:, i], color=colors[i])
  #sns.displot(samples[:, i], color=colors[i], kde=True)
ymax = plt.ylim()[1]
for i in range(3):
  plt.vlines(true_w[i], 0, ymax, color=colors[i])
plt.ylim(0, ymax)
plt.show()

Diagnostics

Trace plots are nice, but diagnostics are nicer!

First we need to run multiple chains. This is as simple as giving a batch of
`initial_state` tensors.

In [ ]:
# Instead of a single set of initial w's, we create a batch of 8.
num_chains = 8
initial_state = np.zeros([num_chains, num_features + 1])

chains, kernel_results = run_chain(initial_state)

r_hat = tfp.mcmc.potential_scale_reduction(chains)
print("Acceptance rate:", kernel_results.inner_results.is_accepted.numpy().mean())
print("R-hat diagnostic (per latent variable):", r_hat.numpy())

In [ ]:
## Sampling the noise scale

# Define the joint_log_prob function, and our unnormalized posterior.
def joint_log_prob(w, sigma, x, y):
  # Our model in maths is
  #   w ~ MVN([0, 0, 0], diag([1, 1, 1]))
  #   y_i ~ Normal(w @ x_i, noise_scale),  i=1..N

  rv_w = tfd.MultivariateNormalDiag(
    loc=np.zeros(num_features + 1),
    scale_diag=np.ones(num_features + 1))
  
  rv_sigma = tfd.LogNormal(np.float64(1.), np.float64(5.))

  rv_y = tfd.Normal(f(x, w), sigma[..., np.newaxis])
  return (rv_w.log_prob(w) +
          rv_sigma.log_prob(sigma) +
          tf.reduce_sum(rv_y.log_prob(y), axis=-1))

In [ ]:
# Create our unnormalized target density by currying x and y from the joint.
def unnormalized_posterior(w, sigma):
  return joint_log_prob(w, sigma, xs, ys)

In [ ]:
# Create an HMC TransitionKernel
hmc_kernel = tfp.mcmc.HamiltonianMonteCarlo(
  target_log_prob_fn=unnormalized_posterior,
  step_size=np.float64(.1),
  num_leapfrog_steps=4)

In [ ]:
# Create a TransformedTransitionKernl
transformed_kernel = tfp.mcmc.TransformedTransitionKernel(
    inner_kernel=hmc_kernel,
    bijector=[tfb.Identity(),    # w
              tfb.Invert(tfb.Softplus())])   # sigma

In [ ]:
# Apply a simple step size adaptation during burnin
@tf.function
def run_chain(initial_state, num_results=1000, num_burnin_steps=500):
  adaptive_kernel = tfp.mcmc.SimpleStepSizeAdaptation(
      transformed_kernel,
      num_adaptation_steps=int(.8 * num_burnin_steps),
      target_accept_prob=np.float64(.75))

  return tfp.mcmc.sample_chain(
    num_results=num_results,
    num_burnin_steps=num_burnin_steps,
    current_state=initial_state,
    kernel=adaptive_kernel,
    seed=(0, 1),
    trace_fn=lambda cs, kr: kr)

In [ ]:
# Instead of a single set of initial w's, we create a batch of 8.
num_chains = 8
initial_state = [np.zeros([num_chains, num_features + 1]),
                 .54 * np.ones([num_chains], dtype=np.float64)]

chains, kernel_results = run_chain(initial_state)

r_hat = tfp.mcmc.potential_scale_reduction(chains)
print("Acceptance rate:", kernel_results.inner_results.inner_results.is_accepted.numpy().mean())
print("R-hat diagnostic (per w variable):", r_hat[0].numpy())
print("R-hat diagnostic (sigma):", r_hat[1].numpy())

In [ ]:
w_chains, sigma_chains = chains

# Trace plots of w (one of 8 chains)
colors = ['b', 'g', 'r', 'teal']
fig, axes = plt.subplots(4, num_chains, figsize=(4 * num_chains, 8))
for j in range(num_chains):
  for i in range(3):
    ax = axes[i][j]
    ax.plot(w_chains[:, j, i], c=colors[i], alpha=.3)
    ax.hlines(true_w[i], 0, 1000, zorder=4, color=colors[i], label="$w_{}$".format(i))
    ax.legend(loc='upper right')
  ax = axes[3][j]
  ax.plot(sigma_chains[:, j], alpha=.3, c=colors[3])
  ax.hlines(noise_scale, 0, 1000, zorder=4, color=colors[3], label=r"$\sigma$".format(i))
  ax.legend(loc='upper right')
fig.tight_layout()
plt.show()

In [ ]:
# Histogram of samples of w
fig, axes = plt.subplots(4, num_chains, figsize=(4 * num_chains, 8))
for j in range(num_chains):
  for i in range(3):
    ax = axes[i][j]
    sns.distplot(w_chains[:, j, i], color=colors[i], norm_hist=True, ax=ax, hist_kws={'alpha': .3})
    #sns.displot(w_chains[:, j, i], color=colors[i], ax=ax , kde=True)
  for i in range(3):
    ax = axes[i][j]
    ymax = ax.get_ylim()[1]
    ax.vlines(true_w[i], 0, ymax, color=colors[i], label="$w_{}$".format(i), linewidth=3)
    ax.set_ylim(0, ymax)
    ax.legend(loc='upper right')

  ax = axes[3][j]
  sns.distplot(sigma_chains[:, j], color=colors[3], norm_hist=True, ax=ax, hist_kws={'alpha': .3})
  #sns.displot(sigma_chains[:, j], color=colors[3], kde=True)
  ymax = ax.get_ylim()[1]
  ax.vlines(noise_scale, 0, ymax, color=colors[3], label=r"$\sigma$".format(i), linewidth=3)
  ax.set_ylim(0, ymax)
  ax.legend(loc='upper right')
fig.tight_layout()
plt.show()